In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
def convert_list_to_in_sql(src):
    return str(src).replace("[", "(").replace("]", ")")

bigquery-public-data.stackoverflow.posts_questions

In [3]:
# Connect to stackoverflow dataset
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
stackOverflow = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                         dataset_name="stackoverflow")

Using Kaggle's public dataset BigQuery integration.


In [4]:
# Get bq assistant for data
bq_assistant = BigQueryHelper("bigquery-public-data", "stackoverflow")

Using Kaggle's public dataset BigQuery integration.


In [5]:
# Select random users, user(id, display_name, creation_date, last_access_date, reputation)
myusers = bq_assistant.head("users", num_rows = 10000).sample(n = 15)[["id", "display_name", "creation_date", "last_access_date"]]

# Id list of my users
id_of_myusers = myusers["id"].values.tolist()
in_sql_id_of_myusers = convert_list_to_in_sql(id_of_myusers)

RetryError: Deadline of 120.0s exceeded while calling target function, last exception: 429 GET https://dp.kaggle.net/bigquery/v2/projects/bigquery-public-data/datasets/stackoverflow?prettyPrint=false: Quota exceeded

Selecting posts, comments, votes:

* posts | posts_questions
* comments | comments + posts_answers + stackoverflow_posts
* votes | votes : votes.post_id in (posts_answers.id + stackoverflow_posts.id = posts : posts.owner_user_id = users.id)

In [7]:
# Select all posts-questions of my users (will be used as posts)
posts_questions_of_myusers = bq_assistant.query_to_pandas("SELECT id, title, body, owner_user_id, creation_date, last_activity_date FROM bigquery-public-data.stackoverflow.posts_questions WHERE owner_user_id IN " 
                                                        + in_sql_id_of_myusers)
id_of_posts_questions_of_myusers = posts_questions_of_myusers["id"].values.tolist()
in_sql_id_of_posts_questions_of_myusers = convert_list_to_in_sql(id_of_posts_questions_of_myusers)

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


In [8]:
# Select all comments (will be used as part of comments)
comments_of_myusers = bq_assistant.query_to_pandas("SELECT id, text, creation_date, post_id, user_id FROM bigquery-public-data.stackoverflow.comments " 
                                                 + "WHERE user_id IN " 
                                                 + in_sql_id_of_myusers 
                                                 + " AND post_id IN " 
                                                 + in_sql_id_of_posts_questions_of_myusers)
id_of_comments_of_myusers = comments_of_myusers["id"].values.tolist()

# Select all posts-answers of my users (will be used as part of comments)
posts_answers_of_myusers = bq_assistant.query_to_pandas("SELECT id, title, body, owner_user_id, creation_date, last_activity_date FROM bigquery-public-data.stackoverflow.posts_answers " 
                                                        + "WHERE owner_user_id IN "
                                                        + in_sql_id_of_myusers
                                                        + "AND parent_id IN"
                                                        + in_sql_id_of_posts_questions_of_myusers)
id_of_posts_answers_of_myusers = posts_answers_of_myusers["id"].values.tolist()

# Select all stackoverflow-posts of my users (will be used as part of comments)
posts_stackoverflow_of_myusers = bq_assistant.query_to_pandas("SELECT id, title, body, owner_user_id, creation_date, last_activity_date FROM bigquery-public-data.stackoverflow.stackoverflow_posts " 
                                                              + "WHERE owner_user_id IN "
                                                              + in_sql_id_of_myusers
                                                              + "AND parent_id IN"
                                                              + in_sql_id_of_posts_questions_of_myusers)
id_of_posts_stackoverflow_of_myusers = posts_stackoverflow_of_myusers["id"].values.tolist()

# id overlapping check
print(set(id_of_comments_of_myusers).intersection(set(id_of_posts_answers_of_myusers)))
print(set(id_of_posts_answers_of_myusers).intersection(set(id_of_posts_stackoverflow_of_myusers)))
print(set(id_of_comments_of_myusers).intersection(set(id_of_posts_stackoverflow_of_myusers)))
print(set(id_of_posts_answers_of_myusers) == set(id_of_posts_stackoverflow_of_myusers))

set()
{13449089, 7607811, 5897220, 6093316, 5933062, 17120260, 22659084, 5880846, 7550990, 6026777, 18606751, 16272036, 7925284, 8396840, 9714729, 14657840, 5880752, 5949362, 11135667, 27538743, 6342839, 16563256, 8307901, 5855551, 8090048, 6137412, 26218053, 30313804, 21445454, 6197327, 27241811, 5777493, 26148699, 28089053, 18302943, 18325731, 9171559, 7239407, 28814970, 5765371, 15057532}
set()
False


In [9]:
# Dublicate id
dublicate = list(set(id_of_posts_answers_of_myusers).intersection(set(id_of_posts_stackoverflow_of_myusers)))
in_sql_dublicate = convert_list_to_in_sql(dublicate)

posts_answers_dublicate = bq_assistant.query_to_pandas("SELECT * FROM bigquery-public-data.stackoverflow.posts_answers "
                                                     + "WHERE id IN "
                                                     + in_sql_dublicate
                                                     + " AND owner_user_id IN " 
                                                     + in_sql_id_of_myusers 
                                                     + " AND parent_id IN " 
                                                     + in_sql_id_of_posts_questions_of_myusers)

stackoverflow_posts_dublicate = bq_assistant.query_to_pandas("SELECT * FROM bigquery-public-data.stackoverflow.stackoverflow_posts "
                                                           + "WHERE id IN "
                                                           + in_sql_dublicate
                                                           + " AND owner_user_id IN " 
                                                           + in_sql_id_of_myusers 
                                                           + " AND parent_id IN " 
                                                           + in_sql_id_of_posts_questions_of_myusers)
print(dublicate)
print(posts_answers_dublicate.size)
print(stackoverflow_posts_dublicate.size)

RetryError: Deadline of 120.0s exceeded while calling target function, last exception: 429 GET https://dp.kaggle.net/bigquery/v2/projects/kaggle-161607/queries/b9d2106c-25a0-4e44-82cc-fbb2bab19038?maxResults=0&location=US&prettyPrint=false: Quota exceeded

In [18]:
# Select all comments of my users for users and posts
comments_of_myusers = bq_assistant.query_to_pandas("(SELECT id, text, post_id, user_id, creation_date FROM bigquery-public-data.stackoverflow.comments " 
                                                 + " WHERE user_id IN " 
                                                 + in_sql_id_of_myusers 
                                                 + " AND post_id IN " 
                                                 + in_sql_id_of_posts_questions_of_myusers + ")"
                                                 + " UNION DISTINCT " 
                                                 + "(SELECT id, body, parent_id, owner_user_id, creation_date FROM bigquery-public-data.stackoverflow.posts_answers " 
                                                 + " WHERE owner_user_id IN " 
                                                 + in_sql_id_of_myusers 
                                                 + " AND parent_id IN " 
                                                 + in_sql_id_of_posts_questions_of_myusers + ")"
                                                 + " UNION DISTINCT "
                                                 + "(SELECT id, body, parent_id, owner_user_id, creation_date FROM bigquery-public-data.stackoverflow.stackoverflow_posts " 
                                                 + " WHERE owner_user_id IN " 
                                                 + in_sql_id_of_myusers 
                                                 + " AND parent_id IN " 
                                                 + in_sql_id_of_posts_questions_of_myusers
                                                 + " AND id NOT IN "
                                                 + in_sql_dublicate + ")")
                                                 
id_of_comments_of_myusers = comments_of_myusers["id"].values.tolist()
print(comments_of_myusers.size)
print(len(set(comments_of_myusers["id"].values.tolist())))

340
68


In [ ]:
votes = bq_assistant.query_to_pandas(
"""
WITH entities AS (
    SELECT id, owner_user_id FROM bigquery-public-data.stackoverflow.posts_questions
    UNION DISTINCT
    SELECT id, owner_user_id FROM bigquery-public-data.stackoverflow.posts_answers 
)
SELECT * FROM bigquery-public-data.stackoverflow.votes 
INNER JOIN posts
ON votes.post_id = posts.id
INNER JOIN bigquery-public-data.stackoverflow.users
ON posts.owner_user_id = users.id
"""
)

In [ ]:
# Interesting fact: votes DO NOT HAVE link to user. So votes are anonymous!!!

# So I decided to get 1451 random votes from first 10000 votes
votes = bq_assistant.head("votes", num_rows = 10000).sample(n = 1451)[["id", "vote_type_id", "creation_date"]]

# Renaming columns
votes.rename(columns = {"creation_date" : "creation_timestrmpz"}, inplace = True)

In [ ]:
# Renaming columns
myusers.rename(columns = {"display_name": "name", "creation_date": "registration_timestrmpz", "last_access_date": "last_access_timestrmpz"}, inplace = True
posts_questions_of_myusers.rename(columns = {"owner_user_id": "user_creator_id", "creation_date": "creation_timestrmpz", "last_activity_date": "last_activity_timestrmpz"}, inplace = True)
comments_of_myusers.rename(columns = {"text": "body", "user_id": "user_creator_id", "creation_date": "creation_timestrmpz"}, inplace = True)

# Exporting to csv
myusers.to_csv("users.csv", index = False)
posts_questions_of_myusers.to_csv("posts.csv", index = False)
comments_of_myusers.to_csv("comments.csv", index = False)
votes.to_csv("votes.csv", index = False)

In [ ]:
a = bq_assistant.table_schema("posts_questions")
b = bq_assistant.table_schema("posts_answers")
a == b

In [ ]:
a

In [ ]:
b